In [135]:
import pandas as pd
import numpy as np

In [136]:
pl_matches = pd.read_csv('../data/pl_matches.csv')
print(pl_matches.info())
pl_matches.value_counts('match_id')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830 entries, 0 to 829
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          830 non-null    object 
 1   time          830 non-null    object 
 2   match_id      830 non-null    object 
 3   league_name   830 non-null    object 
 4   league_id     830 non-null    int64  
 5   opponent      830 non-null    object 
 6   opponent_id   830 non-null    object 
 7   home_away     830 non-null    object 
 8   result        830 non-null    object 
 9   gf            830 non-null    float64
 10  ga            830 non-null    float64
 11  attendance    830 non-null    object 
 12  captain       830 non-null    object 
 13  formation     830 non-null    object 
 14  referee       830 non-null    object 
 15  home_team     830 non-null    object 
 16  home_team_id  830 non-null    object 
dtypes: float64(2), int64(1), object(14)
memory usage: 110.4+ KB
None


match_id
0050df89    1
006fb5b5    1
007b352e    1
00bcfc31    1
00ea7906    1
           ..
fec3438b    1
fed27aaa    1
fede7f6e    1
ffc59ea8    1
fff671a9    1
Name: count, Length: 830, dtype: int64

In [137]:
# pl_matches = pl_matches[pl_matches['league_id'] == 9]
# pl_matches.to_csv('../data/pl_matches.csv', index=False)

# Remove duplicates
pl_matches = pl_matches.drop_duplicates(subset=['match_id', 'home_team'], keep='first')
pl_matches.reset_index(drop=True, inplace=True)
print(pl_matches.info())
pl_matches.value_counts('match_id')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830 entries, 0 to 829
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          830 non-null    object 
 1   time          830 non-null    object 
 2   match_id      830 non-null    object 
 3   league_name   830 non-null    object 
 4   league_id     830 non-null    int64  
 5   opponent      830 non-null    object 
 6   opponent_id   830 non-null    object 
 7   home_away     830 non-null    object 
 8   result        830 non-null    object 
 9   gf            830 non-null    float64
 10  ga            830 non-null    float64
 11  attendance    830 non-null    object 
 12  captain       830 non-null    object 
 13  formation     830 non-null    object 
 14  referee       830 non-null    object 
 15  home_team     830 non-null    object 
 16  home_team_id  830 non-null    object 
dtypes: float64(2), int64(1), object(14)
memory usage: 110.4+ KB
None


match_id
0050df89    1
006fb5b5    1
007b352e    1
00bcfc31    1
00ea7906    1
           ..
fec3438b    1
fed27aaa    1
fede7f6e    1
ffc59ea8    1
fff671a9    1
Name: count, Length: 830, dtype: int64

In [138]:
# Derive new columns
pl_matches['goal_diff'] = pl_matches['gf'] - pl_matches['ga']
pl_matches['points'] = pl_matches['result'].map({'W': 3, 'D': 1, 'L': 0})

In [139]:
pl_matches = pl_matches.sort_values(by=['home_team', 'date'], ascending=[True, True])
pl_matches.reset_index(drop=True, inplace=True)
pl_matches.head()

,date,time,match_id,league_name,league_id,opponent,opponent_id,home_away,result,gf,ga,attendance,captain,formation,referee,home_team,home_team_id,goal_diff,points
0,2023-08-21,20:00,3b5ecd36,Premier League,9,Crystal Palace,47c64c55,Away,W,1.0,0.0,"24,189",Martin Ødegaard,4-3-3,David Coote,Arsenal,18bb7c10,1.0,3
1,2023-08-26,15:00,d8f8f8ad,Premier League,9,Fulham,fd962109,Home,D,2.0,2.0,"59,961",Martin Ødegaard,4-3-3,Paul Tierney,Arsenal,18bb7c10,0.0,1
2,2023-09-24,14:00,08947a10,Premier League,9,Tottenham,361ca564,Home,D,2.0,2.0,"60,156",Martin Ødegaard,4-3-3,Robert Jones,Arsenal,18bb7c10,0.0,1
3,2023-09-30,15:00,8de4aca0,Premier League,9,Bournemouth,4ba7cbea,Away,W,4.0,0.0,"11,193",Martin Ødegaard,4-3-3,Michael Salisbury,Arsenal,18bb7c10,4.0,3
4,2023-11-04,17:30,6096abaa,Premier League,9,Newcastle Utd,b2b47a98,Away,L,0.0,1.0,"52,194",Jorginho,4-3-3,Stuart Attwell,Arsenal,18bb7c10,-1.0,0


In [140]:
# Create rolling stats
pl_matches['avg_goals_last_5'] = pl_matches.groupby('home_team')['gf'].transform(lambda x: x.shift().rolling(window=5, min_periods=1).mean())
pl_matches['avg_goals_against_last_5'] = pl_matches.groupby('home_team')['ga'].transform(lambda x: x.shift().rolling(window=5, min_periods=1).mean())
pl_matches['avg_goals_points_last_5'] = pl_matches.groupby('home_team')['points'].transform(lambda x: x.shift().rolling(window=5, min_periods=1).mean())

In [141]:
def weighted_form_window(x):
    # x is already the rolling window Series
    n = len(x)
    weights = np.arange(1, n+1)  # increasing weights for older → newer
    return np.average(x, weights=weights)

pl_matches['form_score'] = (
    pl_matches.groupby('home_team')['points']
      .transform(lambda x: x.shift().rolling(5, min_periods=1).apply(weighted_form_window, raw=True))
)

In [142]:
# reorder columns
pl_matches = pl_matches[['date', 'home_team', 'opponent', 'avg_goals_last_5', 'avg_goals_against_last_5', 'avg_goals_points_last_5', 'form_score', 'result']]
pl_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830 entries, 0 to 829
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      830 non-null    object 
 1   home_team                 830 non-null    object 
 2   opponent                  830 non-null    object 
 3   avg_goals_last_5          805 non-null    float64
 4   avg_goals_against_last_5  805 non-null    float64
 5   avg_goals_points_last_5   805 non-null    float64
 6   form_score                706 non-null    float64
 7   result                    830 non-null    object 
dtypes: float64(4), object(4)
memory usage: 52.0+ KB


In [143]:
pl_matches.fillna(0, inplace=True)
pl_matches = pd.get_dummies(pl_matches, columns=['home_team', 'opponent'], drop_first=True)

In [144]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
pl_matches['result'] = le.fit_transform(pl_matches['result']) # W=2, D=0, L=1
pl_matches

,date,avg_goals_last_5,avg_goals_against_last_5,avg_goals_points_last_5,form_score,result,home_team_Aston Villa,home_team_Bournemouth,home_team_Brentford,home_team_Brighton,...,opponent_Manchester City,opponent_Manchester Utd,opponent_Newcastle Utd,opponent_Nott'ham Forest,opponent_Sheffield Utd,opponent_Southampton,opponent_Sunderland,opponent_Tottenham,opponent_West Ham,opponent_Wolves
0,2023-08-21,0.000000,0.000000,0.000000,0.000000,2,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2023-08-26,1.000000,0.000000,3.000000,0.000000,0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2023-09-24,1.500000,1.000000,2.000000,0.000000,0,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,2023-09-30,1.666667,1.333333,1.666667,0.000000,2,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2023-11-04,2.250000,1.000000,2.000000,0.000000,1,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,2025-04-26,1.000000,1.200000,1.400000,1.533333,2,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
826,2025-05-25,1.600000,0.600000,2.000000,2.066667,0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
827,2025-08-16,1.400000,0.800000,1.600000,1.733333,1,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
828,2025-09-27,1.200000,1.200000,1.600000,1.200000,0,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [145]:
pl_matches.to_csv('../data/pl_clean.csv', index=False)